<img style="float: center;" src="https://raw.githubusercontent.com/actinia-org/actinia-core/main/docs/docs/actinia_logo.svg" width="25%">

## Einführung

Die cloudbasierte Geoprocessing-Plattform [actinia](https://actinia.mundialis.de/)
ist in der Lage, große Mengen von Geodaten in der Cloud zu verarbeiten und zu
analysieren. Die Software vereinigt viele [GRASS GIS](https://grass.osgeo.org/)
Verarbeitungswerkzeuge und Datenbanken als
[REST-Dienst](https://en.wikipedia.org/wiki/Representational_State_Transfer).
Daher ist der Zugriff auf GRASS-Ressourcen wie Rasterkarten, Raum-Zeit-Rasterdatensätze,
Verarbeitungs- und Analysemodule über URLs möglich. Darüber hinaus ermöglicht
actinia die cloudbasierte Verarbeitung von Daten, zum Beispiel alle Landsat 4-9 Szenen
sowie alle Sentinel-2-Szenen in einer ephemeren Datenbank. Die Berechnungsergebnisse
stehen in der ephemeren Verarbeitung über Objektspeicher als GeoTIFF/COG Rasterdateien oder
GeoPackage-Vektordateien zur Verfügung.

Der actinia-Dienst besteht aus dem *[actinia core](https://github.com/actinia-org/actinia-core)*,
der den grundlegenden, aber anspruchsvollen Verarbeitungsdienst bereitstellt, und
*[actinia plugins](https://github.com/orgs/actinia-org/repositories?q=actinia+plugins&type=all&language=&sort=)*,
die themenpezifische Dienste wie NDVI-Berechnungen aus Sentinel-2 oder Landsat-Daten,
räumlich-zeitliche statistische Analysen und vieles mehr anbieten.

Das folgende Beispiel ist eine überarbeitete Jupyter Notebook Version des Online
[actinia Tutorials](https://actinia-dev.mundialis.de/tutorial/introduction.html).

### Was ist REST?

Der Representational State Transfer ([REST](https://de.wikipedia.org/wiki/Representational_State_Transfer))
ist eine Abstraktion der Struktur und des Verhaltens des World Wide Web
([HTTP](https://de.wikipedia.org/wiki/Hypertext_Transfer_Protocol)). Ziel von REST
ist es, einen Architekturstil zu schaffen, der den Anforderungen des modernen Web
besser gerecht wird. Dabei unterscheidet sich REST von anderen Architekturstilen
vor allem durch die Forderung nach einer einheitlichen Schnittstelle. Der Fokus von
REST liegt auf der Maschine-zu-Maschine-Kommunikation. REST benutzt die
"[request methods](https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Request_methods)"
GET, DELETE, POST und PUT, um Ressourcen mit zustandslosen Operationen zu manipulieren
und zu empfangen.

Während GET-Anfragen einfach von einem Browser gesendet werden können, sind POST-,
PUT- oder DELETE-Anfragen nicht möglich. Um das volle Potenzial von actinia zu nutzen,
benötigen Sie einen HTTP-Client, der alle HTTP-Kommunikationsmethoden beherrscht.
Für diesen Zweck werden wir hier dieses Jupyter Notebook verwenden.

### GRASS GIS Data Management

GRASS GIS, was intern neben GDAL und anderen Komponenten von actinia verwendet wird, hat die in der Abbildung dargestellte Datenbankstruktur.

![grass_format](images/grass_format.png)

**Funktionalität über GRASS GIS hinaus**

Actinia ist nicht nur eine REST-Schnittstelle zu GRASS GIS, sondern bietet die Möglichkeit, seine Funktionalität mit anderer Software (ESA SNAP, GDAL, ...) zu erweitern. Um andere als GRASS GIS Software zu integrieren, muss ein Wrapper-Skript geschrieben werden (Stil: als GRASS GIS Addon Python-Skript), das dann die entsprechenden Funktionsaufrufe der zu integrierenden Software enthält. Der Aufruf von Shell-Befehlen in einer actinia-Prozesskette ist ebenfalls möglich, aber aufgrund von Sicherheitsrisiken eingeschränkt.


### Persistent vs. User Database

Unter **Persistentem Speicher** versteht man einen Datenspeicher, der die Daten auch bei einem Stromausfall und ohne geplante Löschzeiten aufbewahrt. Im Geo/EO-Kontext wird der persistente Speicher z. B. für die Bereitstellung von Basiskartographie verwendet, d. h. für Höhenmodelle, Straßennetze, Gebäudegrundrisse usw.

Der **ephemere Speicher** wird für auf Anfrage berechnete Ergebnisse verwendet, einschließlich benutzergenerierter Daten und temporärer Daten, die in Verarbeitungsketten anfallen. In einem ephemeren Speicher werden die Daten nur für einen begrenzten Zeitraum aufbewahrt (z. B. in Actinia standardmäßig für 24 Stunden).

Im Kontext des Cloud Computing sind diese Unterschiede relevant, da bei der Speicherung von Daten Kosten anfallen.

Dementsprechend bietet actinia zwei Betriebsmodi an: persistente und ephemere (User) Verarbeitung. Insbesondere wird der **actinia server** typischerweise auf einem Server mit Zugriff auf eine persistente GRASS GIS Datenbank (PDB) und optional auf eine oder mehrere GRASS GIS Nutzerdatenbanken (UDB) eingesetzt.

---

### actinia API documentation

Die Schnittstellen Dokumentation findet sich hier:

* [actinia "stable" API v3 docs](https://redocly.github.io/redoc/?url=https://actinia.mundialis.de/api/v3/swagger.json)
* [actinia "development" API v3 docs](https://redocly.github.io/redoc/?url=https://actinia-dev.mundialis.de/api/v3/swagger.json)

---

### Anforderungen

#### Software & Module

Dieses Tutorial setzt voraus, dass Sie mit der Programmiersprache [Python](https://python.org) vertraut sind. Die Kenntnis grundlegender REST-API-Konzepte und deren Verwendung wird ebenfalls vorausgesetzt.

Die in diesem Tutorium verwendeten Python-Module sind:
* [requests](http://docs.python-requests.org/)
* [json](https://docs.python.org/3/library/json.html)
* [leafmap](https://leafmap.org/)


#### Actinia API Benutzer und Passwort

Für diese Demo werden die Anmeldedaten für die Authentifizierung benötigt, die unten in **Vorbereitung** als Variable festgelegt sind. Eine andere Actinia-Instanz kann andere Anmeldedaten erfordern.

### Hilfsmodule und Funktionen

Bevor wir mit dem actinia-Server über Python interagieren, werden wir die erforderlichen Pakete importieren und Hilfsfunktion erstellen.

## Vorbereitung

Um diese Sitzung im Jupyter Notebook zu initialisieren, laden wir einige Python Bibliotheken. `matplotlib` und `leafmap` muss zudem zunächst noch installiert werden.

In [ ]:
!pip install matplotlib
!pip install -U leafmap

Im jupyter hub müssen danach unter Umständen Pfade neu gesetzt werden, damit `matplotlib` auch gefunden wird. Eventuelle Warnungen von `pip install` müssen berücksichtigt werden.

In [ ]:
# read the warnings of pip install to get the paths not in path

import os

path = os.getenv("PATH")
path = path + os.pathsep + "/home/jovyan/.local/bin"
os.environ["PATH"] = path

# adjust PYTHONPATH

import sys
sys.path.append("/home/jovyan/.local/lib/python3.9/site-packages")

Note: In den Notebooks in diesem Workshop wird `matplotlib` zur einfachen Darstellung der Ergebnisse genutzt. Alternativ gibt es die Bibliothek `leafmap`, ein Python-Paket für raumbezogene Analysen und interaktive Kartierungen in einer Jupyter-Umgebung: https://leafmap.org/ (siehe auch: [actinia with leafmap notebook](https://github.com/actinia-org/actinia-python-client/blob/main/notebooks/actinia_leafmap.ipynb)).

In [ ]:
# Zuerst werden die erforderlichen Python Bibliotheken importiert.

import json
import shutil
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import leafmap
import requests
from requests.auth import HTTPBasicAuth

Um die Kommunikation mit dem actinia Server zu vereinfachen, speichern wir die Anmeldedaten und die URL des REST-Servers in Variablen:

In [ ]:
# variables to set the actinia host, version, and user

actinia_baseurl = "https://actinia.mundialis.de"
actinia_version = "v3"
actinia_user = "fossgis2024"
actinia_pw = "ahd5ohghaitiiyoh0hu6"
actinia_url = f"{actinia_baseurl}/api/{actinia_version}"
actinia_auth = HTTPBasicAuth(actinia_user, actinia_pw)

Hilfsfunktionen: \
Funktion um ein formatiertes JSON auszugeben \
Funktion um actinia Job abzufragen

In [ ]:
# helper function to print formatted JSON using the json module

def print_as_json(data):
    print(json.dumps(data, indent=2))

# helper function to verify a request
def verify_request(request, success_code=200):
    if request.status_code != success_code:
        print("ERROR: actinia processing failed with status code %d!" % request.status_code)
        print("See errors below:")
        print_as_json(request.json())
        request_url = request.json()["urls"]["status"]
        requests.delete(url=request_url, auth=actinia_auth)
        raise Exception("The resource <%s> has been terminated." % request_url)

## Beispiele

* Datenverwaltung
* Benutzerdefinierte Verarbeitung: eigene Prozesskette anhand des Beispiels der Berechnung des Normalized Difference Vegetation Index (NDVI).


### Datenverwaltung

Auflistung aller GRASS locations, die über den Endpunkt `/locations` in der persistenten Datenbank von actinia verfügbar sind:

In [ ]:
# make a GET request to the actinia data API
request_url = f"{actinia_url}/locations"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Available locations:")

# print formatted JSON
print_as_json(jsonResponse)

#### Mapsets in Locations auflisten

Auflisten aller **Mapsets** innerhalb der Location `fossgis2024_epsg25832_utm32N` über den Endpunkt `/locations/<location_name>/mapsets`:

In [ ]:
# make a GET request to the actinia data API
request_url = actinia_url + "/locations/fossgis2024_epsg25832_utm32N/mapsets"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Mapsets in fossgis2024_epsg25832_utm32N location:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

#### Inhalt einer Mapset auflisten

Auflisten aller **Rasterkarten** in der Location `fossgis2024_epsg25832_utm32N` und Mapset `PERMANENT` über den Endpunkt `/locations/<location_name>/mapsets/<mapset>/raster_layers`:

In [ ]:
# make a GET request to the actinia data API
request_url = f"{actinia_url}/locations/fossgis2024_epsg25832_utm32N/mapsets/PERMANENT/raster_layers"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Raster layers in mapset PERMANENT of location fossgis2024_epsg25832_utm32N:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

Abfrage der **Rasterinformationen** von der Karte `nDOM` über den Endpunkt `/locations/<location_name>/mapsets/<mapset>/raster_layers/<raster>`:

In [ ]:
# make a GET request to the actinia data API
request_url = f"{actinia_url}/locations/fossgis2024_epsg25832_utm32N/mapsets/PERMANENT/raster_layers/ndom"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Raster info of ndom in mapset PERMANENT of location fossgis2024_epsg25832_utm32N:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

**Rendere eine Rasterkarte**: Rendern der Karte `ndom` über den Endpunkt `/locations/<location_name>/mapsets/<mapset>/raster_layers/<raster>/render`:

In [ ]:
# make a GET request to the actinia data API
request_url = f"{actinia_url}/locations/fossgis2024_epsg25832_utm32N/mapsets/PERMANENT/raster_layers/ndom/render"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth, stream=True)

# check if anything went wrong
verify_request(request, 200)

# download ndom into new folder
os.makedirs('result', exist_ok=True)
with open('result/ndom.png', 'wb') as out_file:
    shutil.copyfileobj(request.raw, out_file)

Die `ndom` Karte kann entweder über den obigen link angezeigt werden (user: `fossgis2024`,password: `ahd5ohghaitiiyoh0hu6`) oder durch Ausführen der folgenden Code-Zelle

In [ ]:
img = np.asarray(Image.open("result/ndom.png"))
imgplot = plt.imshow(img)
plt.xticks([]);
plt.yticks([]);

Auflistung aller **Vektorkarten** in der Location `fossgis2024_epsg25832_utm32N` und Mapset `PERMANENT` über Endpunkt `/locations/<location_name>/mapsets/<mapset>/vector_layers`:

In [ ]:
# make a GET request to the actinia data API
request_url = f"{actinia_url}/locations/fossgis2024_epsg25832_utm32N/mapsets/PERMANENT/vector_layers"
print("actinia GET request:")
print(request_url)
print("---")
request = requests.get(url=request_url, auth=actinia_auth)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()

print("Vector layers in mapset PERMANENT of location fossgis2024_epsg25832_utm32N:")

# print formatted JSON
print_as_json(jsonResponse["process_results"])

---

### Benutzerdefinierte Verarbeitung

Actinia nutzt den **Prozesskettenansatz**, um für Import, Verarbeitung und Export von Geodaten mit dem actinia GRASS GIS Verarbeitungssystem zu kommunizieren.

#### Was ist eine Prozesskette?

Eine Prozesskette ist eine Liste von GRASS GIS Modulen, die nacheinander in der Reihenfolge der Liste ausgeführt werden. GRASS GIS Module werden als Prozessdefinitionen spezifiziert, die den Namen des Befehls, die Ein- und Ausgaben, einschließlich Import- und Exportdefinitionen sowie die Modulflags enthalten.

Die Prozesskette muss in JSON formuliert werden.

#### Ephemere vs. persistente Verarbeitung

Die Verarbeitung erfolgt immer in einer temporären ephemeren Datenbank, mit der die Daten aus der persistenten und der Benutzerdatenbank verknüpft werden.

Der Prozess kann dann **ephemeral** oder **persistent** durchgeführt werden.

Bei der **persistenten** Verarbeitung kann die ephemere Datenbank in die persistente Benutzerdatenbank verschoben werden, so dass die Berechnungsergebnisse in weiteren Verarbeitungsschritten verwendet oder mit Hilfe der actinia REST-Aufrufe visualisiert werden können.

Bei der **ephemeren** Verarbeitung wird die ephemere Datenbank nach der Berechnung entfernt.

Allerdings können alle Raster- und Vektordaten, die während der Verarbeitung erzeugt wurden, mit GDAL/OGR-spezifischen Datentypen exportiert und in einem Objektspeicher außerhalb der actinia-Umgebung gespeichert werden. 

Bei beiden Verarbeitungstypen besteht nur Lesezugriff auf alle Karten des verwendeten persistenten Datenbankspeichers.

#### Erstellen einer Prozesskette Schritt für Schritt

Wir erstellen nun Schritt für Schritt eine Prozesskette anhand des Beispiels der Berechnung des Normalized Difference Vegetation Index (NDVI).

Erstellen Sie zunächst eine leere Prozesskette:

In [ ]:
process_chain = {"version": 1, "list": []}

Fügen Sie den ersten Eintrag in die Prozesskettenliste ein.

Für Rasteroperationen müssen wir die **Rechenregion** ([computational region](https://grasswiki.osgeo.org/wiki/Computational_region)) auf die Region von Interesse mit der gewünschten Auflösung setzen.

***Hinweis:*** Sie müssen den Namen der Karte und der Mapset folgendermaßen angeben: "map_name@mapset_name".

In [ ]:
# list item for g.region
region_process = {
  "id": "g_region_to_dop",
  "module": "g.region",
  "inputs": [
      {
          "param": "raster",
          "value": "dop_nir@PERMANENT"
      }
  ],
  "flags": "p"
}
process_chain["list"].append(region_process)
print_as_json(process_chain)

Nun fügen wir die NDVI-Verarbeitung in die Prozesskettenliste ein:

In [ ]:
# list item for r.mapcalc
ndvi_process = {
  "id": "r_mapcalc_ndvi",
  "module": "r.mapcalc",
  "inputs": [
      {
          "param": "expression",
          "value": "ndvi = round(127.5 * (1.0 + float((dop_nir@PERMANENT - dop_red@PERMANENT) / (dop_nir@PERMANENT + dop_red@PERMANENT))))"
      }
  ]
}
process_chain["list"].append(ndvi_process)
print_as_json(process_chain)

Ausgabe der Statistikinformationen der berechneten NDVI-Karte als Prozessergebnis über `stdout`:

In [ ]:
# list item for r.univar
result_id = "ndvi_stats"
ndvi_stats_process = {
          "id": "r_univar_ndvi",
          "module": "r.univar",
          "inputs": [
              {
                  "param": "map",
                  "value": "ndvi"
              }
          ],
          "flags": "g",
          "stdout": {"id": result_id, "format": "kv", "delimiter": "="}
      }
process_chain["list"].append(ndvi_stats_process)
print_as_json(process_chain)

Export der NDVI-Rasterkarte im COG-Format, da wir im ephemeralen Modus gerechnet haben:

In [ ]:
# list item for exporter
export_process = {
  "id": "exporter_ndvi",
  "module": "exporter",
  "outputs": [
    {
      "export": {
        "type": "raster",
        "format": "COG"
      },
      "param": "map",
      "value": "ndvi"
    }
  ]
}
process_chain["list"].append(export_process)
print_as_json(process_chain)

Führen Sie nun den Job aus, indem Sie die Prozesskette an den ephemeren Endpunkt `/locations/<location_name>/processing_async_export` senden.

In [ ]:
# create a POST request to the Actinia Data API
request_url = f"{actinia_url}/locations/fossgis2024_epsg25832_utm32N/processing_async_export"
request = requests.post(url=request_url, auth=actinia_auth, json=process_chain)

# check if anything went wrong
verify_request(request, 200)

# get a json-encoded content of the response
jsonResponse = request.json()
print(f"Response with status code: {request.status_code}")

# print formatted JSON
print_as_json(jsonResponse)

# status url
request_url = jsonResponse["urls"]["status"]
print(f"status url: {request_url}")

Das Wichtigste an der Actinia-Antwort ist der **Status**, der `accepted` oder `running` sein sollte, und die **Status-URL** unter `urls - status`.

Die Verarbeitung erfolgt asynchron, was bedeutet, dass die Anfrage gesendet wird und Sie nur die Status-URL erhalten, so dass Sie den aktuellen Status der Verarbeitung abfragen können, bis der Job beendet ist.

Der **Status** eines Prozesses kann sein:
* accepted: actinia hat den Auftrag erhalten und wird in Kürze mit der Verarbeitung beginnen
* running: actinia führt den Auftrag aus
* finished: actinia hat den Auftrag erfolgreich beendet
* error: während der Ausführung des Auftrags ist ein Fehler aufgetreten
* terminated: ein: Benutzer:in hat den Auftrag abgebrochen

Job abfragen, bis zum Abschluss oder Fehler:

In [ ]:
# continue polling until finished
print(request_url)

while request.status_code == 200 and \
        jsonResponse["message"] != "Processing successfully finished":
    request = requests.get(url=request_url, auth=actinia_auth)
    jsonResponse = request.json()

# check if anything went wrong
verify_request(request, 200)

# print formatted JSON
print_as_json(jsonResponse)

`ndvi`-Ergebniskarte anzeigen, die Ergebniskarten-URL ist in der Antwort enthalten und hier muss noch `user:password@server` eingefügt werden `exported_raster`.

In [ ]:
result_url = jsonResponse["urls"]["resources"][0]
print(result_url)
raster_url = result_url.replace("//", f"//{actinia_user}:{actinia_pw}@")

# visualization with leafmap
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
m.add_cog_layer(
    raster_url,
    label="NDVI map",
    colormap_name="greens",
)
# show map
m

In der Prozesskette wurden die NDVI Statistiken als weiteres Prozessergebnisse festgelegt, und können abgefragt werden.

In [ ]:
print("NDVI stats as process results:")
print_as_json(jsonResponse["process_results"][result_id])